In [1]:
from sentence_transformers import CrossEncoder

D:\Projects\venv2\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary D:\Projects\venv2\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


In [29]:
query_1 = "How many people live in Berlin?"
passages_1 = [
    "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.",
    "Berlin has a yearly total of about 135 million day visitors, making it one of the most-visited cities in the European Union.",
    "In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.",
]

query_2 = "Explain the effects of railroads in Britain"
passages_2 = [
    "Railroads revolutionized transport and trade in Britain...",
    "The industrial revolution and railroads helped...",
    "Some unrelated text here..."
]

query_3 = "تاثیرات راه‌آهن‌ها در اقتصاد ایران چیست؟"

passages_3 = [
    "راه‌آهن‌ها باعث تسهیل حمل و نقل و توسعه اقتصادی در ایران شدند.",
    "صنعت نفت در ایران تاثیر بسیار زیادی بر اقتصاد داشت.",
    "ورزش فوتبال در ایران محبوبیت زیادی دارد و تأثیر فرهنگی زیادی داشته است.",
    "راه‌آهن‌ها در دوران پهلوی نقش مهمی در گسترش تجارت و حمل‌ونقل ایفا کردند."
]

In [ ]:
from sentence_transformers import CrossEncoder

# 1. Load a pretrained CrossEncoder model
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")
# reranker = CrossEncoder("cross-encoder/ms-marco-xlm-roberta-base")

query = query_1
passages = passages_1


# 2a. Either predict scores pairs of texts
# scores = reranker.predict([(query, passage) for passage in passages])
# print(scores)

# 2b. Or rank a list of passages for a query
ranks = reranker.rank(query, passages, return_documents=True)

print("Query:", query)
for rank in ranks:
    print(f"- #{rank['corpus_id']} ({rank['score']:.2f}): {rank['text']}")

Query: How many people live in Berlin?
- #0 (8.61): Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.
- #2 (6.35): In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.
- #1 (5.51): Berlin has a yearly total of about 135 million day visitors, making it one of the most-visited cities in the European Union.


'\nQuery: How many people live in Berlin?\n- #0 (8.61): Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.\n- #2 (6.35): In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.\n- #1 (5.51): Berlin has a yearly total of about 135 million day visitors, making it one of the most-visited cities in the European Union.\n'

Qwen3

In [38]:
from sentence_transformers import CrossEncoder

# Load the sequence classification reranker (no architecture mismatch)
reranker = CrossEncoder("tomaarsen/Qwen3-Reranker-0.6B-seq-cls")

# Prepare your query and passage candidates as before
pairs = [
    ("Explain the effects of railroads in Britain", doc)
    for doc in passages
]
scores = reranker.predict(pairs)


ValueError: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [ ]:
from sentence_transformers import CrossEncoder

# Load the classification reranker
reranker = CrossEncoder("tomaarsen/Qwen3-Reranker-0.6B-seq-cls")

query = "Explain the effects of railroads in Britain"
candidate_docs = [
    "Railroads revolutionized transport and trade in Britain...",
    "The industrial revolution and railroads helped...",
    "Some unrelated text here..."
]

pairs = [(query, doc) for doc in candidate_docs]
scores = reranker.predict(pairs)
ranked = [doc for _, doc in sorted(zip(scores, candidate_docs), reverse=True)]

print("Scores:", scores)
print("Ranked docs:", ranked)


ValueError: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Reranker-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-Reranker-0.6B")

Exception: data did not match any variant of untagged enum ModelWrapper at line 757479 column 3

In [35]:
# Requires transformers>=4.51.0
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

def format_instruction(instruction, query, doc):
    if instruction is None:
        instruction = 'Given a web search query, retrieve relevant passages that answer the query'
    output = "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(instruction=instruction,query=query, doc=doc)
    return output

def process_inputs(pairs):
    inputs = tokenizer(
        pairs, padding=False, truncation='longest_first',
        return_attention_mask=False, max_length=max_length - len(prefix_tokens) - len(suffix_tokens)
    )
    for i, ele in enumerate(inputs['input_ids']):
        inputs['input_ids'][i] = prefix_tokens + ele + suffix_tokens
    inputs = tokenizer.pad(inputs, padding=True, return_tensors="pt", max_length=max_length)
    for key in inputs:
        inputs[key] = inputs[key].to(model.device)
    return inputs

@torch.no_grad()
def compute_logits(inputs, **kwargs):
    batch_scores = model(**inputs).logits[:, -1, :]
    true_vector = batch_scores[:, token_true_id]
    false_vector = batch_scores[:, token_false_id]
    batch_scores = torch.stack([false_vector, true_vector], dim=1)
    batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
    scores = batch_scores[:, 1].exp().tolist()
    return scores

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-Reranker-0.6B", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-Reranker-0.6B").eval()
# We recommend enabling flash_attention_2 for better acceleration and memory saving.
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-Reranker-0.6B", torch_dtype=torch.float16, attn_implementation="flash_attention_2").cuda().eval()
token_false_id = tokenizer.convert_tokens_to_ids("no")
token_true_id = tokenizer.convert_tokens_to_ids("yes")
max_length = 8192

prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
prefix_tokens = tokenizer.encode(prefix, add_special_tokens=False)
suffix_tokens = tokenizer.encode(suffix, add_special_tokens=False)
        
task = 'Given a web search query, retrieve relevant passages that answer the query'

queries = ["What is the capital of China?",
    "Explain gravity",
]

documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
]

pairs = [format_instruction(task, query, doc) for query, doc in zip(queries, documents)]

# Tokenize the input texts
inputs = process_inputs(pairs)
scores = compute_logits(inputs)

print("scores: ", scores)


Exception: data did not match any variant of untagged enum ModelWrapper at line 757479 column 3

In [32]:
!pip show transformers

Name: transformers
Version: 4.43.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: d:\projects\venv2\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: auto_gptq, autoawq, compressed-tensors, lm_eval, optimum, peft, sentence-transformers, tokenicer, TTS, vllm


Alibaba

In [50]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name_or_path = "Alibaba-NLP/gte-multilingual-reranker-base"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path, trust_remote_code=True,
    torch_dtype=torch.float16
    )
model.eval()

NewForSequenceClassification(
  (new): NewModel(
    (embeddings): NewEmbeddings(
      (word_embeddings): Embedding(250048, 768, padding_idx=1)
      (rotary_emb): NTKScalingRotaryEmbedding()
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): NewEncoder(
      (layer): ModuleList(
        (0-11): 12 x NewLayer(
          (attention): NewSdpaAttention(
            (qkv_proj): Linear(in_features=768, out_features=2304, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (o_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (mlp): NewGatedMLP(
            (up_gate_proj): Linear(in_features=768, out_features=6144, bias=False)
            (down_proj): Linear(in_features=3072, out_features=768, bias=True)
            (act_fn): GELUActivation()
            (hidden_dropout): Dropout(p=0.1, inplace=Fals

In [51]:
query_1 = "How many people live in Berlin?"
passages_1 = [
    "Berlin had a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.",
    "Berlin has a yearly total of about 135 million day visitors, making it one of the most-visited cities in the European Union.",
    "In 2013 around 600,000 Berliners were registered in one of the more than 2,300 sport and fitness clubs.",
]

query_2 = "Explain the effects of railroads in Britain"
passages_2 = [
    "Railroads revolutionized transport and trade in Britain...",
    "The industrial revolution and railroads helped...",
    "Some unrelated text here..."
]

query_3 = "تاثیرات راه‌آهن‌ها در اقتصاد ایران چیست؟"

passages_3 = [
    "راه‌آهن‌ها باعث تسهیل حمل و نقل و توسعه اقتصادی در ایران شدند.",
    "صنعت نفت در ایران تاثیر بسیار زیادی بر اقتصاد داشت.",
    "ورزش فوتبال در ایران محبوبیت زیادی دارد و تأثیر فرهنگی زیادی داشته است.",
    "راه‌آهن‌ها در دوران پهلوی نقش مهمی در گسترش تجارت و حمل‌ونقل ایفا کردند."
]

In [56]:
query = query_3
passages = passages_3

pairs = [(query, doc) for doc in passages]
pairs

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([ 2.3125, -0.7656, -3.3184, -0.2749])


Cohere

In [49]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder("tomaarsen/Qwen3-Reranker-0.6B-seq-cls")

scores = reranker.predict([
    ("Explain the effects of railroads in Britain", "Railroads revolutionized transport..."),
    ("Explain the effects of railroads in Britain", "The industrial revolution and railroads helped..."),
])
print("Scores:", scores)


ValueError: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.